In [78]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [79]:
inputFile = 'shakespeare.txt'
with open(inputFile) as f:
  text = f.read()
  print('length of dataset in characters: ', len(text))

length of dataset in characters:  1115393


In [80]:
# this is the vocabulary i.e. all teh unique characters
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [81]:
# create a mapping from characters to integers (i.e. encoding and decoding)
# this is needed as the tensors work with numbers

# create two lookup dictionaries from char to int, and vice versa
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

test = encode('Hello World!')
print(test)
print(decode(test))

[20, 43, 50, 50, 53, 1, 35, 53, 56, 50, 42, 2]
Hello World!


In [82]:
# encode the whole input text and store it as a tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)

In [83]:
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115393]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [84]:
# split data into training and valdiation sets
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]
block_size = 8
train_data[:block_size + 1] # we need +1 to get 8 training sets
 

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [85]:
x = train_data[:block_size] # the training input
y = train_data[1:block_size + 1] # the resulting test (one offset compared to train)
for t in range(block_size):
  context = x[:t + 1]
  target = y[t]
  print(f'when input is {context} the target is: {target}')

when input is tensor([18]) the target is: 47
when input is tensor([18, 47]) the target is: 56
when input is tensor([18, 47, 56]) the target is: 57
when input is tensor([18, 47, 56, 57]) the target is: 58
when input is tensor([18, 47, 56, 57, 58]) the target is: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is: 58


In [86]:
torch.manual_seed(13371337)
batch_count = 4  # how many sequences will be processed in time i.e. how much training can we do in parallell afaik
block_size = 8 # how large is the context i.e. how many chars can we look back at and use in our prediction

def get_batches(data, block_size, batch_count):
  ix = torch.randint(len(data) - block_size, (batch_count,)) 
  x = torch.stack([data[i:i + block_size] for i in ix]) # training input
  y = torch.stack([data[i + 1:i+block_size + 1] for i in ix]) # target output
  return x, y

xb, yb = get_batches(train_data, block_size, batch_count)
print('inputs: ')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)
print('-------')

for b in range(batch_count):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f'for context {context.tolist()} the target is {target}')


inputs: 
torch.Size([4, 8])
tensor([[46, 39, 58,  1, 51, 47, 57, 41],
        [39, 47, 42,  0, 20, 47, 57,  1],
        [ 1, 42, 43, 39, 58, 46,  6,  1],
        [59, 50, 58, 12,  0, 21, 58,  1]])
targets:
torch.Size([4, 8])
tensor([[39, 58,  1, 51, 47, 57, 41, 39],
        [47, 42,  0, 20, 47, 57,  1, 58],
        [42, 43, 39, 58, 46,  6,  1, 39],
        [50, 58, 12,  0, 21, 58,  1, 52]])
-------
for context [46] the target is 39
for context [46, 39] the target is 58
for context [46, 39, 58] the target is 1
for context [46, 39, 58, 1] the target is 51
for context [46, 39, 58, 1, 51] the target is 47
for context [46, 39, 58, 1, 51, 47] the target is 57
for context [46, 39, 58, 1, 51, 47, 57] the target is 41
for context [46, 39, 58, 1, 51, 47, 57, 41] the target is 39
for context [39] the target is 47
for context [39, 47] the target is 42
for context [39, 47, 42] the target is 0
for context [39, 47, 42, 0] the target is 20
for context [39, 47, 42, 0, 20] the target is 47
for context [

In [87]:
#create the bigram language model
import torch
import torch.nn as nn
from torch.nn import functional as F
import math

torch.manual_seed(13371337)


class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()

    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, contexts, targets=None):

    # we make a prediction based on a single token
    logits = self.token_embedding_table(contexts) # Batch Time Channel tensor (BTC)

    if targets is None:
      return logits, None
    
    # we have provided targets so compute the loss
    # we need to reshape the data for use in cross entropy
    B, T, C = logits.shape
    logits = logits.view(B * T, C)
    targets = targets.view(B * T)

    # compare to the target using neg log likeliehood loss
    loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    # idx is (B, T) array of incdices into the current context
    for _ in range(max_new_tokens):
      logits, loss = self.forward(idx)
      logits = logits[:, -1, :] # pick the last element from the time dimension B, _T_, C
      probs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1) # samples from the probabilities
      idx = torch.cat((idx, idx_next), dim=1) # add the prediction to the time dimension
    return idx

m = BigramLanguageModel(vocab_size)
out, loss = m.forward(xb, yb)
print(out.shape)
print("expected loss", -math.log(1/vocab_size))
print(loss)

# lets generate some text
# this text will be completely random as the model is not trained
newline = torch.zeros((1, 1), dtype=torch.long)
output = m.generate(idx = newline, max_new_tokens=100)
print(decode(output[0].tolist()))

torch.Size([32, 65])
expected loss 4.174387269895637
tensor(4.7554, grad_fn=<NllLossBackward0>)

CvOXxfSWjg--,aavU.fsW3jsoq:IH!U!RFPsUc:OHOnSVBMulr;ReeeHPT&Uv'MOE,s;oBgFZqee.3gJ:SKVTudIZqfCFfHOzWfC


In [88]:
@torch.no_grad()
def estimate_loss(model, data, block_size, batch_count, eval_iters):
  out = {}
  model.eval()
  losses = torch.zeros(eval_iters)
  for k in range(eval_iters):
    X, Y = get_batches(data, block_size, batch_count)
    logits, loss = model.forward(X, Y)
    losses[k] = loss.item()
  model.train() # why this one?
  return losses.mean()
      

In [98]:
def train_model(model, train_data, val_data):
  optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
  batch_size = 32
  max_iters = 5000
  eval_iters = 200
  eval_interval = 200
  for steps in range(max_iters):
    xb, yb = get_batches(train_data, block_size, batch_size)
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    if steps % eval_interval == 0:
      train_loss = estimate_loss(model, train_data, block_size, batch_size, eval_iters)
      val_loss = estimate_loss(model, val_data, block_size, batch_size, eval_iters)
      print(f"step {steps}: train loss {train_loss:.4f}, val loss {val_loss:.4f}")



step 0: train loss 2.4711, val loss 2.4925
step 200: train loss 2.4666, val loss 2.4829
step 400: train loss 2.4649, val loss 2.4882
step 600: train loss 2.4632, val loss 2.4874
step 800: train loss 2.4623, val loss 2.4859
step 1000: train loss 2.4580, val loss 2.4762
step 1200: train loss 2.4678, val loss 2.4924
step 1400: train loss 2.4577, val loss 2.4813
step 1600: train loss 2.4536, val loss 2.4793
step 1800: train loss 2.4573, val loss 2.4856
step 2000: train loss 2.4608, val loss 2.4865
step 2200: train loss 2.4549, val loss 2.4756
step 2400: train loss 2.4601, val loss 2.4879
step 2600: train loss 2.4559, val loss 2.4792
step 2800: train loss 2.4581, val loss 2.4890
step 3000: train loss 2.4533, val loss 2.4860
step 3200: train loss 2.4576, val loss 2.4857
step 3400: train loss 2.4529, val loss 2.4829
step 3600: train loss 2.4554, val loss 2.4872
step 3800: train loss 2.4549, val loss 2.4828
step 4000: train loss 2.4544, val loss 2.4749
step 4200: train loss 2.4706, val loss 2.

In [ ]:
train_model(m, train_data, val_data)

In [90]:
newline = torch.zeros((1, 1), dtype=torch.long)
output = m.generate(idx = newline, max_new_tokens=200)
print(decode(output[0].tolist()))


RD:
Bursowers, m, qkethe thememe? F goug wid pr?
Be be k bet thithispy did:
Tho
AS pe:
I
ps PRY authatwheleind ckntxTCENBO; wh oZEO:
Thase by CHe Icorinavelur edrdimar' s 'l us, befrd tr s wooamonthe 


In [103]:
# we now add some attention. This is done by using the average of the previous token values
# this is a simplification and much information is lost but better than bi-gram

class BigramLanguageModel_v2(nn.Module):
  def __init__(self, vocab_size, block_size, n_embed):
    super().__init__()

    self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
    self.position_embedding_table = nn.Embedding(block_size, n_embed)
    self.lm_head = nn.Linear(n_embed, vocab_size)

  def forward(self, contexts, targets=None):
    B, T = contexts.shape

    tok_emb = self.token_embedding_table(contexts)
    pos_emb = self.position_embedding_table(torch.arange(T, device='cpu'))
    x = tok_emb + pos_emb

    logits = self.lm_head(x) # Batch Time Channel tensor (BTC)

    if targets is None:
      return logits, None
    
    # we have provided targets so compute the loss
    # we need to reshape the data for use in cross entropy
    B, T, C = logits.shape
    logits = logits.view(B * T, C)
    targets = targets.view(B * T)

    # compare to the target using neg log likeliehood loss
    loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    # idx is (B, T) array of incdices into the current context
    for _ in range(max_new_tokens):
      idx_cond = idx[:, -block_size:]
      logits, loss = self.forward(idx_cond)
      logits = logits[:, -1, :] # pick the last element from the time dimension B, _T_, C
      probs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1) # samples from the probabilities
      idx = torch.cat((idx, idx_next), dim=1) # add the prediction to the time dimension
    return idx


In [104]:
mv2 = BigramLanguageModel_v2(vocab_size, block_size, 32)
train_model(mv2, train_data, val_data)

step 0: train loss 4.4758, val loss 4.4705
step 200: train loss 3.0527, val loss 3.0695
step 400: train loss 2.7814, val loss 2.8115
step 600: train loss 2.6903, val loss 2.7051
step 800: train loss 2.6368, val loss 2.6456
step 1000: train loss 2.5944, val loss 2.6118
step 1200: train loss 2.5682, val loss 2.5878
step 1400: train loss 2.5571, val loss 2.5770
step 1600: train loss 2.5508, val loss 2.5612
step 1800: train loss 2.5339, val loss 2.5562
step 2000: train loss 2.5276, val loss 2.5470
step 2200: train loss 2.5289, val loss 2.5360
step 2400: train loss 2.5085, val loss 2.5226
step 2600: train loss 2.5164, val loss 2.5260
step 2800: train loss 2.4999, val loss 2.5350
step 3000: train loss 2.5026, val loss 2.5162
step 3200: train loss 2.4990, val loss 2.5193
step 3400: train loss 2.5029, val loss 2.5206
step 3600: train loss 2.5028, val loss 2.5049
step 3800: train loss 2.4894, val loss 2.5032
step 4000: train loss 2.4940, val loss 2.5082
step 4200: train loss 2.4909, val loss 2.

In [111]:
output = mv2.generate(idx = newline, max_new_tokens=200)
print(decode(output[0].tolist()))



BRK:

Fak foanisand c'llle s,

WELA!


Andanpre my lhe eseseve my ed terd s, thimy or erins d lllth tht-e we t sherhoug bored thf eathere errsefy hede tht wen, co l d ththithe'dy bap ll t ME n n he t
